In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://ftp.wayne.edu/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

sc = spark.sparkContext

--2020-05-18 06:50:21--  http://ftp.wayne.edu/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving ftp.wayne.edu (ftp.wayne.edu)... 141.217.0.199
Connecting to ftp.wayne.edu (ftp.wayne.edu)|141.217.0.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232530699 (222M) [application/x-gzip]
Saving to: ‘spark-2.4.5-bin-hadoop2.7.tgz’

spark-2.4.5-bin-had 100%[===================>] 221.76M  33.0MB/s    in 7.9s    

2020-05-18 06:50:29 (27.9 MB/s) - ‘spark-2.4.5-bin-hadoop2.7.tgz’ saved [232530699/232530699]



In [0]:
iprdd=sc.textFile('/content/drive/My Drive/CSE545_Project/Education.csv')

In [0]:
def customAppend(str,myList):
    if len(str) > 0:
        myList.append(str)
    return myList

def customSplit(str, delim, ignoreBegin, ignoreEnd):
    myList = []
    currString = ''
    isIgnore = False
    isIgnoreJustEnded = False
    for i in str:
        if i == ignoreBegin and not isIgnore:
            customAppend(currString, myList)
            currString = ignoreBegin
            isIgnore = True
        elif i == ignoreEnd and isIgnore:
            currString = currString + ignoreEnd
            myList.append(currString)
            currString = ''
            isIgnore = False
            isIgnoreJustEnded = True
        elif isIgnore:
            currString = currString + i
        elif i == delim:
            # customAppend(currString, myList)
            if isIgnoreJustEnded:
                isIgnoreJustEnded = False
            else:
                myList.append(currString)
            currString = ''
        else:
            currString = currString + i
    customAppend(currString, myList)
    return myList

In [0]:
def intConv(x):
  op=[]
  try:
    for i in x[1]:
      if '"' not in i:
        op.append(float(i))
      else:
        op.append(int(i[1:-1].replace(',','')))
  except:
    op=[]
  return (x[0],op)

In [0]:
rdd1=iprdd.filter(lambda x: 'F' not in x[0]).map(lambda x:customSplit(x,',','\"','\"')).map(lambda x: (int(x[0]),x[-4:])).filter(lambda x: x[0]!=0)

In [0]:
rdd1.map(lambda x: intConv(x)).filter(lambda x: len(x[1])>2).map(lambda x:(x[0],(1,x[1]))).saveAsTextFile('/content/drive/My Drive/CSE545_Project/OP/edu')

In [0]:
Uiprdd=sc.textFile('/content/drive/My Drive/CSE545_Project/Unemployment.csv')

In [0]:
Urdd1=Uiprdd.filter(lambda x: 'F' not in x[0]).map(lambda x:customSplit(x,',','\"','\"')).map(lambda x: (int(x[0]),x[-3:-1])).filter(lambda x: x[0]!=0)

In [0]:
Urdd1.map(lambda x: intConv(x)).filter(lambda x: len(x[1])==2).map(lambda x:(x[0],(2,x[1]))).saveAsTextFile('/content/drive/My Drive/CSE545_Project/OP/unemp')

In [0]:
import os
op=[]
f = open("/content/drive/My Drive/CSE545_Project/MOP/merged.txt", "w")
for i in os.listdir('/content/drive/My Drive/CSE545_Project/OP'):
    for j in os.listdir('/content/drive/My Drive/CSE545_Project/OP/'+str(i)):
      if '.' not in j:
        for k in open('/content/drive/My Drive/CSE545_Project/OP/'+str(i)+'/'+str(j)):
          f.write(k)



In [0]:
miprdd=sc.textFile('/content/drive/My Drive/CSE545_Project/MOP/merged.txt')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
lendict={1:4,2:2,3:1,4:1,5:1,6:1,7:2,8:1,9:1}
def filterfunc(x):
  op=True
  try:
    for i in x[1]:
      if lendict[i[0]]!=len(i[1]):
        op=False
  except:
    op=False
  return op
  
oplen=9
def appenderfunc(x):
  op=[[]for i in range(oplen)]
  Yval=[]
  for i in x[1]:
    if i[0]==100:
      Yval=i[1]
    else:
      op[i[0]-1]=i[1] if isinstance(i[1],list) else [i[1]]
    

  flat_list = []
  for sublist in op:
      for elem in sublist:
          flat_list.append(elem)
  return (int(x[0]),(flat_list,Yval))

def remFunc(x):
  try: 
    k=int(x[0])
    return True
  except:
    return False

In [0]:
mrdd1=miprdd.map(lambda x: eval(x)).filter(lambda x: remFunc(x)).map(lambda x: (int(x[0]),[x[1]])).reduceByKey(lambda a,b:a+b)

In [0]:
mrdd2=mrdd1.map(lambda x: appenderfunc(x)).filter(lambda x: len(x[1][0])==14 and len(x[1][1])==1)

In [0]:
def floatConvTest(X):
  try:
    for i in range(len(X[1][0])):
      X[1][0][i]=float(X[1][0][i])
    for i in range(len(X[1][1])):
      X[1][1][i]=float(X[1][1][i])
    return True
  except:
    return False

def floatConv(X):
  for i in range(len(X[1][0])):
    X[1][0][i]=float(X[1][0][i])
  for i in range(len(X[1][1])):
    X[1][1][i]=float(X[1][1][i])
    return (X[0],X[1])



In [0]:
mrdd3=mrdd2.filter(lambda x: floatConvTest(x)).map(lambda x: floatConv(x))

In [0]:
broaddict=mrdd3.collect()
broadcastdict={}
for i in broaddict:
  broadcastdict[i[0]]=i[1]

In [0]:
broad_map=sc.broadcast(broadcastdict)

In [0]:
xy_vals=mrdd3.map(lambda x:(1,[x[1]])).reduceByKey(lambda a,b: a+b).collect()

In [0]:
X=[]
Y=[]
for xy in xy_vals[0][1]:
  X.append(xy[0])
  Y.append(xy[1])

In [0]:
import numpy as np

X_n=np.asarray(X,np.float32)
Y_n=np.asarray(Y,np.float32)

#added
mean_dict_x={}
std_dict_x={}
x_mean_list = []
x_std_list = []
#added ends

ymean=np.mean(Y_n)
ystd=np.std(Y_n)
Y_n=(Y_n-ymean)/ystd

Xtrans=X_n.transpose()
for j in range(len(Xtrans)):
  xmean=np.mean(Xtrans[j])
  mean_dict_x[j]=xmean
  xstd=np.std(Xtrans[j])
  std_dict_x[j]=xstd
  Xtrans[j]=(Xtrans[j]-xmean)/xstd
X_n=Xtrans.transpose()

X_trans=X_n.transpose()
sq=X_trans.dot(X_n)

beta=(np.linalg.inv(sq)).dot(X_trans.dot(Y_n))

mean_broadcast=sc.broadcast(mean_dict_x)
std_broadcast=sc.broadcast(std_dict_x)
y_broadcast_factor=sc.broadcast((ymean,ystd))

In [0]:
def simFind(key,x_n,y,num):
  x=x_n+y
  ref_x_n=broad_map.value[num][0]
  ref_y_n=broad_map.value[num][1]
  ref_x=ref_x_n+ref_y_n
  xsq=0
  ref_xsq=0
  op=0
  for i in range(len(ref_x)):
    ref_xsq+=ref_x[i]**2
    xsq+=x[i]**2
    op+=ref_x[i]*x[i]
  return (key,op/(ref_xsq*xsq))



In [0]:
# GET TOP 20 SIMILAR COUNTY'S with reference to FIPS code 36103(Suffolk county)
sim_list=mrdd3.map(lambda x: simFind(x[0],x[1][0],x[1][1],6071)).collect()

In [0]:
sorted_sim_list=sorted(sim_list,key=(lambda x: x[1]),reverse=True)

In [0]:
sorted_sim_list[:20]

[(1005, 6.072201935727749e-12),
 (1119, 6.024952129321313e-12),
 (5123, 5.993215703313667e-12),
 (5107, 5.99134912669114e-12),
 (1087, 5.967032274110777e-12),
 (1093, 5.889068650345702e-12),
 (1109, 5.888310703888595e-12),
 (1053, 5.8317911126981844e-12),
 (5005, 5.783701355421822e-12),
 (1131, 5.675507300290871e-12),
 (4023, 5.659967341419728e-12),
 (1059, 5.6451690118855725e-12),
 (1061, 5.632212629879488e-12),
 (1039, 5.55374877412518e-12),
 (1017, 5.531125256463546e-12),
 (5071, 5.472896335424373e-12),
 (5103, 5.368607953963509e-12),
 (5111, 5.359706804318036e-12),
 (1123, 5.28779672051156e-12),
 (1071, 5.2349288525481795e-12)]

In [0]:
def findData(record,targetFips):
  if record[0]==targetFips:
    return True
  return False
targetFips = 36107
targetRdd = mrdd3.filter(lambda x:findData(x,targetFips))

In [0]:
targetRdd.collect()[0][1]

([9.2,
  36.1,
  29.2,
  25.5,
  4.1,
  60309.0,
  48203.0,
  0.0,
  63.0,
  414.9119349418086,
  9.7,
  2.7423352026,
  19.537902318218396,
  22280.77090637512],
 [10.2])

In [0]:
x_values = np.asarray(targetRdd.collect()[0][1][0],np.float32)
#increase schools in county by 10
tot_schools=(x_values[6]/1000000)*x_values[9]
x_normalize=[]

for i in range(len(x_values)):
  k=(x_values[i]-mean_broadcast.value[i])/std_broadcast.value[i]
  x_normalize.append(k)

x_normalize=np.asarray(x_normalize,np.float32)
y_value = x_normalize.dot(np.asarray(beta))

x_values[9]=((tot_schools+30)/x_values[6])*1000000
x_values[1]+=6
x_normalize=[]

for i in range(len(x_values)):
  k=(x_values[i]-mean_broadcast.value[i])/std_broadcast.value[i]
  x_normalize.append(k)

x_normalize=np.asarray(x_normalize,np.float32)
y_value_new = x_normalize.dot(np.asarray(beta))



In [0]:
#change in poverty
y_value_new-y_value

In [0]:
([9.2,
  36.1,
  29.2,
  25.5,
  4.1,
  60309.0,
  48203.0,
  0.0,
  63.0,
  414.9119349418086,
  9.7,
  2.7423352026,
  19.537902318218396,
  22280.77090637512],
 [10.2])

([15.8,
  37.8,
  24.6,
  21.8,
  4.5,
  27859.0,
  12427.0,
  80.46994447573832,
  21.0,
  563.2896113301682,
  27.7,
  10.372885631,
  3.718717152042135,
  9167.28105234566],
 [34.7])



In [0]:
https://drive.google.com/open?id=1qqMyZgPgQ5k_NT4KmIdSR64swfFrjdJT